In [1]:
import pandas as pd
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
import time

from numpy import hstack, vstack
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()

'c:\\Users\\Ansh\\Desktop\\MEST\\HMG3\\Rapid_fin\\deep learning modeling'

In [3]:
csv_add = find_directory(foldername = [], filename = 'SOC_Point_Data.csv')
dat = pd.read_csv(csv_add, index_col = (0,1,2,3,4))

In [4]:
def Get_Data_all(dat) :
    
    RPT_MODE = "0.1C"
    SOC_Range = [9,10,11,12]

    Time_Range = range(4, 1201, 2)
    SOC_Range = [str(i) for i in SOC_Range]
    
    Data = dat

    X = Data.loc[RPT_MODE, SOC_Range]
    Y = Data.loc[RPT_MODE, ["SOH", "Next_SOH", "Ratio_SOH", "Ratio_CYC"]].groupby(level = ["Next", "Path", "Number"]).mean()
    y = Y["Next_SOH"]
    
    return X, y

In [5]:
def Even_Split(X, y, test_size, rs):
    XX = {n: X.xs(key=n, level='Next', drop_level=False) for n in ["M", "D", "H"]}
    yy = {n: y.xs(key=n, level='Next', drop_level=False) for n in ["M", "D", "H"]}

    XXX, yyy = {n: [] for n in ["M", "D", "H"]}, {n: [] for n in ["M", "D", "H"]}
    for n in ["M", "D", "H"]:
        for path in range(1, 5):
            X_path = XX[n].loc[XX[n].index.get_level_values('Path').str.len() == path]
            y_path = yy[n].loc[yy[n].index.get_level_values('Path').str.len() == path]

            if 'Time' in y_path.index.names:
                y_path = y_path.reset_index('Time', drop=True)

            XXX[n].append(X_path)
            yyy[n].append(y_path)

    XX_tn, XX_te = {n: [] for n in ["M", "D", "H"]}, {n: [] for n in ["M", "D", "H"]}
    yy_tn, yy_te = {n: [] for n in ["M", "D", "H"]}, {n: [] for n in ["M", "D", "H"]}

    for n in ["M", "D", "H"]:
        for path in range(1, 5):
            X_temp = XXX[n][path-1]
            y_temp = yyy[n][path-1]

            cell_index = y_temp.index.drop_duplicates()

            cells_tn, cells_te = train_test_split(cell_index, test_size=test_size, random_state=rs)

            X_tn = X_temp[X_temp.index.droplevel('Time').isin(cells_tn)]
            X_te = X_temp[X_temp.index.droplevel('Time').isin(cells_te)]

            y_tn = y_temp.loc[cells_tn]
            y_te = y_temp.loc[cells_te]

            XX_tn[n].append(X_tn)
            XX_te[n].append(X_te)
            yy_tn[n].append(y_tn)
            yy_te[n].append(y_te)

    X_tn = pd.concat([pd.concat(XX_tn[n]) for n in ["M", "D", "H"]])
    X_te = pd.concat([pd.concat(XX_te[n]) for n in ["M", "D", "H"]])
    y_tn = pd.concat([pd.concat(yy_tn[n]) for n in ["M", "D", "H"]])
    y_te = pd.concat([pd.concat(yy_te[n]) for n in ["M", "D", "H"]])

    return X_tn, X_te, y_tn, y_te

In [6]:
def convert_to_tensor(X_tn, X_te, y_tn, y_te, batch_size=32, val_size=1/6, rs=100):
    """
    X_tn, X_te: MultiIndex DataFrame (Next, Path, Number, Time)
    y_tn, y_te: Series or DataFrame with index (Next, Path, Number)
    """

    X_tr, X_vl, y_tr, y_vl = Even_Split(X_tn, y_tn, val_size, rs)
    keys_train = y_tr.index
    keys_val = y_vl.index

    scaler = StandardScaler()
    X_tn_scaled = X_tn.copy()
    X_te_scaled = X_te.copy()

    scaler.fit(X_tn.values)
    X_tn_scaled.loc[:, :] = scaler.transform(X_tn.values)
    X_te_scaled.loc[:, :] = scaler.transform(X_te.values)

    next_map = {'M': 0, 'D': 1, 'H': 2}
    get_next = lambda idx: torch.nn.functional.one_hot(torch.tensor([next_map[i] for i in idx.get_level_values("Next")]), num_classes=3).float()

    def to_tensor(X_df, y_df, keys):
        X_group = X_df.groupby(level=["Next", "Path", "Number"])
        X_tensor = torch.stack([
            torch.tensor(X_group.get_group(k).values, dtype=torch.float32)
            for k in keys
        ])
        y_tensor = torch.tensor(y_df.loc[keys].values, dtype=torch.float32).unsqueeze(1)
        return X_tensor, y_tensor

    next_train = get_next(pd.MultiIndex.from_tuples(keys_train, names=["Next", "Path", "Number"]))
    next_val   = get_next(pd.MultiIndex.from_tuples(keys_val,   names=["Next", "Path", "Number"]))
    next_test  = get_next(y_te.index)
    
    X_train_tensor, y_train_tensor = to_tensor(X_tn_scaled, y_tn, keys_train)
    X_val_tensor,   y_val_tensor   = to_tensor(X_tn_scaled, y_tn, keys_val)
    X_test_tensor,  y_test_tensor  = to_tensor(X_te_scaled, y_te, y_te.index.drop_duplicates())

    train_loader = DataLoader(TensorDataset(X_train_tensor, next_train, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(TensorDataset(X_val_tensor, next_val,  y_val_tensor),   batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(TensorDataset(X_test_tensor, next_test,  y_test_tensor),  batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor, scaler

In [7]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [8]:
class Trainer:
    def __init__(self, model, lr=1e-3, weight_decay=0, epoch=1000, patience=50, random_seed = 0):
        self.random_seed = random_seed
        setRandomSeed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.criterion = nn.MSELoss()
        self.cri2 = MAPELoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=200, gamma=0.8)
        self.epoch = epoch
        self.patience = patience

        self.train_loss = []
        self.test_loss = []
        self.val_loss = []

    def train(self, train_loader, test_loader=None, val_loader=None):
        best_val_loss = float('inf')
        best_model_state = None
        epochs_no_improve = 0

        for ep in range(1, self.epoch + 1):
            self.model.train()
            for x_batch, onehot, y_batch in train_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(x_batch, onehot)
                loss = self.cri2(outputs, y_batch)
                loss.backward()
                self.optimizer.step()

            self.scheduler.step()

            avg_train_loss = self.evaluate(train_loader)
            test_loss = self.evaluate(test_loader) if test_loader else None
            val_loss = self.evaluate(val_loader) if val_loader else None

            self.train_loss.append(avg_train_loss)
            self.test_loss.append(test_loss)
            self.val_loss.append(val_loss)

            if val_loader:
                if val_loss < best_val_loss - 1e-4:
                    best_val_loss = val_loss
                    best_model_state = self.model.state_dict()
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= self.patience:
                        print(f"Early stopping at epoch {ep}")
                        break

        if best_model_state:
            self.model.load_state_dict(best_model_state)

    def evaluate(self, data_loader):
        if data_loader is None:
            return None
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for x_batch, onehot, y_batch in data_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)
                outputs = self.model(x_batch, onehot)
                loss = self.cri2(outputs, y_batch)
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def predict(self, x, onehot):
        self.model.eval()
        with torch.no_grad():
            x = x.to(self.device)
            onehot = onehot.to(self.device)
            return self.model(x, onehot)

In [9]:
def plot_results(model, info, train_loader, val_loader, test_loader, plot = True):
    rs, l1 = info
    
    y_true_train = []
    y_pred_train = []
    
    y_true_val = []
    y_pred_val = []
    
    y_true_test = []
    y_pred_test = []

    y_true_test_M = []
    y_pred_test_M = []

    y_true_test_D = []
    y_pred_test_D = []

    y_true_test_H = []
    y_pred_test_H = []
    
    for x_batch, onehot, y_batch in train_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_train.append(y_batch)
        y_pred_train.append(preds.cpu())
    
    for x_batch, onehot, y_batch in val_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_val.append(y_batch)
        y_pred_val.append(preds.cpu())
    
    for x_batch, onehot, y_batch in test_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_test.append(y_batch)
        y_pred_test.append(preds.cpu())
    
        onehot_np = onehot.cpu().numpy()
        y_true_np = y_batch.cpu().numpy()
        y_pred_np = preds.cpu().numpy()
    
        for i in range(len(onehot_np)):
            if np.array_equal(onehot_np[i], [1, 0, 0]):  # 'M'
                y_true_test_M.append(y_true_np[i])
                y_pred_test_M.append(y_pred_np[i])
            elif np.array_equal(onehot_np[i], [0, 1, 0]):  # 'D'
                y_true_test_D.append(y_true_np[i])
                y_pred_test_D.append(y_pred_np[i])
            elif np.array_equal(onehot_np[i], [0, 0, 1]):  # 'H'
                y_true_test_H.append(y_true_np[i])
                y_pred_test_H.append(y_pred_np[i])
    
    y_true_train = torch.cat(y_true_train).numpy()
    y_pred_train = torch.cat(y_pred_train).numpy()
    
    y_true_val = torch.cat(y_true_val).numpy()
    y_pred_val = torch.cat(y_pred_val).numpy()
    
    y_true_test = torch.cat(y_true_test).numpy()
    y_pred_test = torch.cat(y_pred_test).numpy()

    y_true_test_M = np.array(y_true_test_M)
    y_pred_test_M = np.array(y_pred_test_M)
    
    y_true_test_D = np.array(y_true_test_D)
    y_pred_test_D = np.array(y_pred_test_D)
    
    y_true_test_H = np.array(y_true_test_H)
    y_pred_test_H = np.array(y_pred_test_H)
    
    mape_M = mean_absolute_percentage_error(y_true_test_M, y_pred_test_M) * 100 if len(y_true_test_M) > 0 else np.nan
    mape_D = mean_absolute_percentage_error(y_true_test_D, y_pred_test_D) * 100 if len(y_true_test_D) > 0 else np.nan
    mape_H = mean_absolute_percentage_error(y_true_test_H, y_pred_test_H) * 100 if len(y_true_test_H) > 0 else np.nan
    
    train_mape = mean_absolute_percentage_error(y_true_train, y_pred_train) * 100
    val_mape = mean_absolute_percentage_error(y_true_val, y_pred_val) * 100
    test_mape = mean_absolute_percentage_error(y_true_test, y_pred_test) * 100
    print(f"Model: {model}, rs: {rs}, l1: {l1}\n Train MAPE: {train_mape:.2f}%, Val MAPE: {val_mape:.2f}%, Test MAPE: {test_mape:.2f}%")
    print(f"Test MAPE by 'Next': M: {mape_M:.2f}%, D: {mape_D:.2f}%, H: {mape_H:.2f}%")
    
    if plot == True:
        _ = plt.figure()
        _ = plt.scatter(y_true_train, y_pred_train, label = 'Train')
        _ = plt.scatter(y_true_val, y_pred_val, label = 'Val')
        _ = plt.scatter(y_true_test, y_pred_test, label = 'Test')
        min_val = min(y_true_train.min(), y_true_test.min())
        max_val = max(y_true_train.max(), y_true_test.max())
        _ = plt.plot([min_val, max_val], [min_val, max_val], 'k--', label='Ideal line')
        _ = plt.xlabel('True SOH')
        _ = plt.ylabel('Predicted SOH')
        _ = plt.legend()
        _ = plt.title(f'Random state: {rs}, hid: {hid}, {nl} layer, lr: {lr}')
    return train_mape, val_mape, test_mape, mape_M, mape_D, mape_H

def plot_loss(info, train_loss, val_loss, test_loss):
    rs, l1 = info
    _ = plt.figure()
    _ = plt.plot(train_loss, label = 'Train loss')
    _ = plt.plot(val_loss, label = 'Val loss')
    _ = plt.plot(test_loss, label = 'Test loss')
    _ = plt.ylim([0, 2])
    _ = plt.xlabel('Epoch')
    _ = plt.ylabel('Loss')
    _ = plt.legend()
    _ = plt.title(f'Random state: {rs}, hid: {hid}, {nl} layers, lr: {lr}')

## EN

In [10]:
class ENEncoder(nn.Module):
    def __init__(self, input_dim, output_dim, l1_ratio=0.5):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim, bias=True)
        self.l1_ratio = l1_ratio

    def forward(self, x):
        return self.linear(x)

    def elasticnet_penalty(self):
        l1 = torch.norm(self.linear.weight, p=1)
        l2 = torch.norm(self.linear.weight, p=2) ** 2
        return self.l1_ratio * l1 + (1 - self.l1_ratio) * l2

In [11]:
class MLP(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=64, num_layers=2, next_dim=3):
        super().__init__()
        total_input = input_dim + next_dim
        layers = [
            nn.Linear(total_input, hidden_dim),
            nn.ReLU()
        ]
        for _ in range(num_layers - 1):
            layers += [nn.Linear(hidden_dim, hidden_dim), nn.ReLU()]
        layers += [nn.Linear(hidden_dim, 1)]
        self.model = nn.Sequential(*layers)

    def forward(self, x_feat, onehot):
        x = torch.cat([x_feat, onehot], dim=1)
        return self.model(x)

In [12]:
class EN_MLP(nn.Module):
    def __init__(self, out_dim=4, hidden_dim=64, num_layers=2, next_dim=3, l1_ratio = 0.1):
        super().__init__()
        self.in_dim = 599*4
        self.out_dim = out_dim
        self.next_dim = next_dim
        self.encoder = ENEncoder(self.in_dim, self.out_dim, l1_ratio)
        self.mlp = MLP(input_dim=out_dim, hidden_dim=hidden_dim, num_layers=num_layers, next_dim=next_dim)

    def forward(self, x, onehot):
        x_flat = x.view(x.size(0), -1)
        x_encoded = self.encoder(x_flat)
        return self.mlp(x_encoded, onehot)

    def elasticnet_penalty(self):
        return self.encoder.elasticnet_penalty()

class MAPELoss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super(MAPELoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + self.epsilon))) * 100

In [13]:
class Trainer_EN:
    def __init__(self, model, lr=1e-3, weight_decay=0, epoch=1000, patience=50, random_seed = 0, alpha = 1e-3):
        self.random_seed = random_seed
        #setRandomSeed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.criterion = nn.MSELoss()
        self.cri2 = MAPELoss()
        self.optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=200, gamma=0.8)
        self.epoch = epoch
        self.patience = patience
        self.alpha = alpha

        self.train_loss = []
        self.test_loss = []
        self.val_loss = []

    def train(self, train_loader, test_loader=None, val_loader=None):
        best_val_loss = float('inf')
        best_model_state = None
        epochs_no_improve = 0

        for ep in range(1, self.epoch + 1):
            self.model.train()
            for x_batch, onehot, y_batch in train_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(x_batch, onehot)
                base_loss = self.criterion(outputs, y_batch)
                reg_loss = self.model.elasticnet_penalty()
                loss = base_loss + self.alpha * reg_loss
                
                loss.backward()
                self.optimizer.step()

            self.scheduler.step()

            avg_train_loss = self.evaluate(train_loader)
            test_loss = self.evaluate(test_loader) if test_loader else None
            val_loss = self.evaluate(val_loader) if val_loader else None

            self.train_loss.append(avg_train_loss)
            self.test_loss.append(test_loss)
            self.val_loss.append(val_loss)

            if val_loader:
                if val_loss < best_val_loss - 1e-4:
                    best_val_loss = val_loss
                    best_model_state = self.model.state_dict()
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= self.patience:
                        print(f"Early stopping at epoch {ep}")
                        break

        if best_model_state:
            self.model.load_state_dict(best_model_state)

    def evaluate(self, data_loader):
        if data_loader is None:
            return None
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for x_batch, onehot, y_batch in data_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)
                outputs = self.model(x_batch, onehot)
                loss = self.cri2(outputs, y_batch)
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def predict(self, x, onehot):
        self.model.eval()
        with torch.no_grad():
            x = x.to(self.device)
            onehot = onehot.to(self.device)
            return self.model(x, onehot)

In [14]:
random_states = [100, 120, 140, 160, 180]
bs = 12
ep = 1000
hid = 16
layer = 5
lr = 1e-3 

l1s = [0.1, 0.3, 0.5, 0.7, 0.9]

results_df = pd.DataFrame(columns = ['rs', 'l1', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE'])

X, y = Get_Data_all(dat)

for rs, l1 in product(random_states, l1s):
    setRandomSeed(rs)
    
    start = time.time()
    X_tn, X_te, y_tn, y_te = Even_Split(X, y, 1/3, rs)
    train_loader, val_loader, test_loader, X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor, X_test_tensor, y_test_tensor, scaler = convert_to_tensor(X_tn, X_te, y_tn, y_te, bs, 1/6, rs)
    
    model = EN_MLP(4, hid, layer, 3, l1)
    trainer = Trainer_EN(model, lr=lr, epoch = ep, random_seed = rs)
    
    trainer.train(train_loader, val_loader, test_loader)

    info = [rs, l1]
    results = plot_results("EN", info, train_loader, val_loader, test_loader, plot = False)

    temp_df = pd.DataFrame(info+list(results)).T
    temp_df.columns = ['rs', 'l1', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']
    results_df = pd.concat([results_df, temp_df])
    print(f"{time.time()-start:.2f} s")

Early stopping at epoch 62
Model: EN, rs: 100, l1: 0.1
 Train MAPE: 2.11%, Val MAPE: 3.13%, Test MAPE: 2.16%
Test MAPE by 'Next': M: 2.25%, D: 1.77%, H: 2.47%
3.52 s
Early stopping at epoch 55
Model: EN, rs: 100, l1: 0.3
 Train MAPE: 1.53%, Val MAPE: 1.28%, Test MAPE: 1.43%
Test MAPE by 'Next': M: 1.57%, D: 1.36%, H: 1.36%
2.06 s
Early stopping at epoch 79
Model: EN, rs: 100, l1: 0.5
 Train MAPE: 1.32%, Val MAPE: 1.14%, Test MAPE: 1.20%
Test MAPE by 'Next': M: 1.23%, D: 1.12%, H: 1.25%
2.85 s
Early stopping at epoch 91
Model: EN, rs: 100, l1: 0.7
 Train MAPE: 1.38%, Val MAPE: 1.02%, Test MAPE: 1.24%
Test MAPE by 'Next': M: 1.25%, D: 1.19%, H: 1.27%
3.28 s
Early stopping at epoch 97
Model: EN, rs: 100, l1: 0.9
 Train MAPE: 1.49%, Val MAPE: 1.20%, Test MAPE: 1.37%
Test MAPE by 'Next': M: 1.47%, D: 1.34%, H: 1.31%
3.55 s
Early stopping at epoch 150
Model: EN, rs: 120, l1: 0.1
 Train MAPE: 1.90%, Val MAPE: 1.71%, Test MAPE: 1.80%
Test MAPE by 'Next': M: 1.67%, D: 1.78%, H: 1.95%
5.34 s
Ear

In [15]:
summary_df = results_df.groupby(['l1'])[['Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']].mean().reset_index()
summary_df
summary_df.to_csv('EN_next_info_sum.csv')
results_df.to_csv('EN_next_info.csv')

,l1,Train MAPE,Val MAPE,Test MAPE,Next M MAPE,Next D MAPE,Next H MAPE
0,0.1,1.738808,1.873823,1.738977,1.780028,1.638202,1.798700
1,0.3,2.046160,2.077415,2.070527,2.095910,2.003427,2.112243
2,0.5,1.644468,1.552144,1.553407,1.603984,1.473725,1.582513
3,0.7,2.001730,1.988089,2.017828,2.025146,1.993603,2.034734
4,0.9,1.837555,1.796024,1.650253,1.626557,1.543016,1.781187
